In [2]:
# !pip install -qU openpyxl

In [3]:
import pandas as pd

dfs = pd.read_excel("../data/quality_annotation.xlsx", sheet_name=None)
for k in dfs:
    dfs[k] = dfs[k].dropna(subset=["id"])
    dfs[k] = dfs[k].astype({"id": int})

In [4]:
df = pd.concat(dfs, keys=dfs.keys(), names=["annotator", "row"])
df = df.iloc[:, :6] # keep 1st 6 columns
df.columns = df.columns.str.split("\n").str[0]
df = df.reset_index(drop=False).drop(columns="row")
df = df.rename(
    columns={
        '"Correctitud"': "correct",
        "No ambigüedad ": "unambiguous",
        "Relevancia": "relevant",
    }
)
df[["correct", "unambiguous", "relevant"]] = df[["correct", "unambiguous", "relevant"]].apply(pd.to_numeric, errors="coerce")
df.head(2)

,annotator,id,query,text,correct,unambiguous,relevant
0,DF,3814522,a dónde pertenece la alcarria,La Alcarria. La Alcarria es una comarca natura...,1.0,1.0,1.0
1,DF,842215,a donde se conecta la pulsera antiestatica,Dispositivo antiestático. Usualmente esta puls...,1.0,1.0,1.0


In [5]:
print(df.dtypes)

annotator       object
id               int64
query           object
text            object
correct        float64
unambiguous    float64
relevant       float64
dtype: object


In [6]:
# Unique queries by annotator:
df.drop_duplicates(["annotator", "query"]).groupby("annotator").size()

annotator
DF     250
IB     250
JMP    200
VC     250
dtype: int64

In [7]:
# Check: there are 50 in common for all annotators
df_tmp = df.drop_duplicates(["annotator", "query"])
df_tmp["query"].value_counts().value_counts()

# Keep the IDs of the queries that are common to all annotators:
queries_tmp = df_tmp["query"].value_counts()
common_queries = queries_tmp[queries_tmp == 4].index
print(len(common_queries))

50


In [8]:
# Keep queries effectively annotated:

# remove all rows with NaN values in any of the 3 columns:
df = df.dropna(subset=["correct", "unambiguous", "relevant"])
# remove all rows with values outside other than [0, 1]:
df = df[(df["correct"].isin([0, 1])) & (df["unambiguous"].isin([0, 1])) & (df["relevant"].isin([0, 1]))]
# remove dups by annotator, id:
df = df.drop_duplicates(subset=["annotator", "id"])

In [9]:
# df["annotator"].value_counts()

In [10]:
df.query("query in @common_queries").groupby("annotator").size()

annotator
DF     50
IB     50
JMP    50
VC     50
dtype: int64

In [11]:
df_tmp = df.drop_duplicates(["annotator", "query"])
df_tmp["query"].value_counts().value_counts()

count
1    655
4     50
Name: count, dtype: int64

In [12]:
# For eac annot, #annot, %correct, %unambiguous, %relevant:
df.groupby("annotator").apply(
    lambda x: pd.Series({
        "#annot": len(x),
        "%correct": 100 * x["correct"].mean(),
        "%unambiguous": 100 * x["unambiguous"].mean(),
        "%relevant": 100 * x["relevant"].mean(),
    })
).reset_index()

,annotator,#annot,%correct,%unambiguous,%relevant
0,DF,204.0,90.686275,87.254902,90.196078
1,IB,250.0,97.600000,91.600000,88.800000
2,JMP,199.0,95.477387,86.432161,86.934673
3,VC,202.0,89.603960,93.069307,93.564356


In [13]:
# to measure rates, use:
# queries annotated by all annotators + sample rest of queries so that each annotator has the same number of queries
# keep the majority vote for each query
mask_common = df.groupby("query")["annotator"].transform("nunique") == df["annotator"].nunique()
df_common = df[mask_common].copy()
df_rest = df[~mask_common].copy()
min_queries = df_rest["annotator"].value_counts().min()
df_rest = (
    df_rest.groupby("annotator")
    .apply(lambda x: x.sample(min_queries, random_state=33, replace=False))
    .reset_index(drop=True)
)
df_tmp = pd.concat([df_common, df_rest], ignore_index=True)
df_rates = df_tmp.groupby(["query", "id"])[
    ["correct", "unambiguous", "relevant"]].mean().round().astype(int).reset_index()

print(df_rates["id"].nunique())
print(df_rates.shape)
df_rates.head(2)

646
(646, 5)


,query,id,correct,unambiguous,relevant
0,a cuantos pasos se coloca la barrera en un tir...,987454,1,1,1
1,a donde iban los muertos en el antiguo testamento,991716,1,1,1


In [14]:
print(f"% correct = {df_rates['correct'].mean() * 100:.1f}")
print(f"% unambiguous = {df_rates['unambiguous'].mean() * 100:.1f}")
print(f"% relevant = {df_rates['relevant'].mean() * 100:.1f}")

% correct = 93.5
% unambiguous = 90.1
% relevant = 90.2


In [15]:
print(f"% correct = {df_rates['correct'].mean() * 100:.1f}")
print(f"% unambiguous = {df_rates['unambiguous'].mean() * 100:.1f}")
print(f"% relevant = {df_rates['relevant'].mean() * 100:.1f}")

% correct = 93.5
% unambiguous = 90.1
% relevant = 90.2


In [16]:
# To measure agreement, keep the queries annotated by all annotators:
mask = df.groupby("query")["annotator"].transform("nunique") == df["annotator"].nunique()
df_common = df[mask].copy()

df_common["annotator"].value_counts()

annotator
DF     50
IB     50
JMP    50
VC     50
Name: count, dtype: int64

In [17]:
df_common.head(2)

,annotator,id,query,text,correct,unambiguous,relevant
3,DF,1009535,a que corriente filosofica pertenece platon,Realismo filosófico. En la filosofía griega an...,1.0,1.0,1.0
11,DF,143694,a quien le quito el marido natalia de la casa ...,Natalia París. Fue pareja durante varios años ...,0.0,0.0,0.0


In [18]:
# df_common.sort_values(["id", "annotator"]).to_csv(
#     "quality_annotation_common.csv", index=False)

In [19]:
# Compute % of times all annotators agree on each category on a given query:
df_tmp = df_common.groupby("id")[["correct", "unambiguous", "relevant"]].mean()
print((df_tmp == 1).mean())
# print(df_tmp)


correct        0.78
unambiguous    0.74
relevant       0.78
dtype: float64
